In [2]:
import os
import numpy as np
import torch
import torch.nn.functional as F
import polyscope as ps
import mcubes

from tqdm.notebook import tqdm

os.chdir('..')
from iarap.model.neural_rtf import NeuralRTF, NeuralRTFConfig
from iarap.model.neural_sdf import NeuralSDF, NeuralSDFConfig
from iarap.utils.meshing import *

ps.init()

In [12]:
SDF_CKPT = 'assets/weights/sdf/dragon.pt'
RTF_CKPT = ''
NUM_PATCH_PTS = 100
PATCH_RADIUS = 0.03
SURFACE_SAMPLES = 100
SPACE_SAMPLES = 0
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


In [4]:
sdf_model: NeuralSDF = NeuralSDFConfig().setup().to(DEVICE)
rtf_model: NeuralRTF = NeuralRTFConfig().setup().to(DEVICE)

if len(SDF_CKPT) > 0:
    sdf_model.load_state_dict(torch.load(SDF_CKPT))

if len(RTF_CKPT) > 0:
    rtf_model.load_state_dict(torch.load(RTF_CKPT))

In [5]:
steps = torch.linspace(-1.0, 1.0, 512, device=DEVICE)
xx, yy, zz = torch.meshgrid(steps, steps, steps, indexing="ij")
volume = torch.vstack([xx.ravel(), yy.ravel(), zz.ravel()]).T.float()
f_eval = []
with torch.no_grad():
    for sample in tqdm(torch.split(volume, 300000, dim=0)):
        f_eval.append(sdf_model(sample.contiguous())['dist'].cpu().numpy())
f_volume = np.concatenate(f_eval, axis=0).reshape(*([512] * 3))
shape_verts, shape_faces = mcubes.marching_cubes(f_volume, 0.0)
shape_verts /= 512 // 2
shape_verts -= 1.0

  0%|          | 0/448 [00:00<?, ?it/s]

## Minor examples

### Surface projection

In [ ]:
initial = torch.tensor([[-0.7, 0.0, 0.0], 
                        [-0.6, 0.1, 0.0],
                        [-0.5, 0.0, 0.3]], device=DEVICE)

trajectory = [initial]
offsets = []

point = initial
for it in range(5):
    point_new = sdf_model.project_nearest(point)
    trajectory.append(point_new)
    offsets.append(point_new - point)
    point = point_new

offsets.append(torch.zeros_like(initial))
trajectory = torch.cat(trajectory, dim=0).view(-1, 3).cpu().detach().numpy()
offsets = torch.cat(offsets, dim=0).view(-1, 3).cpu().detach().numpy()

In [ ]:
ps.register_surface_mesh("Input Shape", shape_verts, shape_faces, enabled=True)
traj_pc = ps.register_point_cloud("Trajectory", trajectory, enabled=True)
traj_pc.add_vector_quantity("Offsets", offsets, vectortype='ambient', enabled=True)
ps.show()
ps.remove_all_structures()

## Local Patch Meshing

### Patch Sampling and Triangulation

In [6]:
patches = {'vert': {}, 'triv': {}}
sampling_methods = {'runif': sphere_random_uniform,
                    'rnorm': sphere_gaussian_radius,
                    'linear': sphere_sunflower,
                    'normal': gaussian_max_norm}

In [13]:
i = 0
for name, sampler in sampling_methods.items():
    patch = get_patch_mesh(sampler, delaunay, NUM_PATCH_PTS, PATCH_RADIUS, DEVICE)
    patches['vert'][name] = patch[0].cpu().numpy()
    patches['vert'][name][:, 0] += PATCH_RADIUS * (2.2 * i)
    patches['triv'][name] = patch[1].cpu().numpy()
    i += 1

In [14]:
for name in sampling_methods.keys():
    ps.register_surface_mesh(f"{name}_patch", patches['vert'][name], patches['triv'][name], enabled=True, edge_width=1)

ps.show()
ps.remove_all_structures()

### Patch Projection

In [ ]:
SELECTED_PATCH = 'runif'

plane_coords, triangles = patches['vert'][SELECTED_PATCH], patches['triv'][SELECTED_PATCH]

In [ ]:
surf_sample = sdf_model.sample_zero_level_set(SURFACE_SAMPLES, 0.05, 10000, (-1, 1), 5).detach()
space_sample = torch.rand(SPACE_SAMPLES, 3, device=DEVICE) * 2 - 1
samples = torch.cat([surf_sample, space_sample], dim=0).detach()

In [ ]:
sdf_outs = sdf_model(samples, with_grad=True)
sample_dist, patch_normals = sdf_outs['dist'], F.normalize(sdf_outs['grad'], dim=-1)
tangent_planes = sdf_model.tangent_plane(samples).cpu().numpy()

In [ ]:
tangent_coords = (np.expand_dims(tangent_planes, 1) @ plane_coords.reshape(1, -1, 3, 1)).squeeze() 
tangent_pts = tangent_coords + samples.unsqueeze(1).detach().cpu().numpy()
triangles_all = np.expand_dims(triangles, 0) + (tangent_pts.shape[1] * np.arange(0, tangent_pts.shape[0]).reshape(-1, 1, 1))

tangent_pts = tangent_pts.reshape(-1, 3)
triangles_all = triangles_all.reshape(-1, 3)

In [ ]:
ps.register_surface_mesh("Input Shape", shape_verts, shape_faces, enabled=True, transparency=0.5)
ps.register_surface_mesh("Projected Patches", tangent_pts, triangles_all, enabled=True, edge_width=1.0)
normals_pc = ps.register_point_cloud("Normal Origins", samples.cpu().detach().numpy(), radius=0.0, enabled=True)
normals_pc.add_vector_quantity("Patch Normals", patch_normals.cpu().detach().numpy(), enabled=True)
ps.show()
ps.remove_all_structures()

### Patch Fitting/Deformation

In [ ]:
level_set_verts = torch.from_numpy(tangent_pts).to(DEVICE, torch.float)
for it in range(5):
    level_set_verts = sdf_model.project_level_sets(level_set_verts, sample_dist)
level_set_verts = level_set_verts.cpu().detach().view(-1, 3).numpy()

In [ ]:
ps.register_surface_mesh("Input Shape", shape_verts, shape_faces, enabled=True, transparency=0.5)
ps.register_surface_mesh("Deformed Patches", level_set_verts, triangles_all, enabled=True, edge_width=1.0)
normals_pc = ps.register_point_cloud("Normal Origins", samples.cpu().detach().numpy(), radius=0.0, enabled=True)
normals_pc.add_vector_quantity("Patch Normals", patch_normals.cpu().detach().numpy(), enabled=True)
ps.show()
ps.remove_all_structures()

### Adaptive Patch Subdivision

In [ ]:
patch_vertices = level_set_verts.view(SURFACE_SAMPLES + SPACE_SAMPLES, NUM_PATCH_PTS, 3)
tri_centers = patch_vertices[:, triangles].mean(dim=-2)
center_sdf = sdf_model(tri_centers)['dist']